In [143]:
import pandas as pd
import matplotlib.pyplot as plt

In [167]:
df = pd.read_csv('/home/max/Documents/train.csv', sep = ';')

In [168]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,47,blue-collar,single,basic.9y,unknown,no,no,cellular,may,wed,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.334,5099.1,no
2995,53,admin.,married,university.degree,no,no,yes,telephone,may,tue,...,13,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
2996,60,retired,married,university.degree,no,yes,no,cellular,oct,mon,...,2,999,0,nonexistent,-1.1,94.601,-49.5,1.000,4963.6,no
2997,32,technician,single,university.degree,no,no,no,cellular,nov,thu,...,1,999,1,failure,-0.1,93.200,-42.0,4.076,5195.8,no


In [169]:
df.drop(['duration', 'nr.employed'], axis = 1, inplace = True)

In [170]:
df_new = df.drop('y', axis = 1)
df_new.columns[df_new.dtypes == 'object'].to_list()

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [171]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [172]:
X, y = df.drop(['y'], axis = 1), pd.get_dummies(df.y).yes

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [174]:
from catboost import Pool, CatBoostClassifier

In [175]:
model = CatBoostClassifier(iterations=13,
                           depth = 7,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [176]:
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=df_new.columns[df_new.dtypes == 'object'].to_list())
test_pool = Pool(X_test, cat_features=df_new.columns[df_new.dtypes == 'object'].to_list()) 

In [177]:
grid = {'iterations': range(2, 20, 2),
        'depth': range(2, 10)
       }

grid_search_result = model.grid_search(grid, 
                                       train_pool)

0:	learn: 0.3054717	test: 0.2850786	best: 0.2850786 (0)	total: 6.69ms	remaining: 6.69ms
1:	learn: 0.2979540	test: 0.2732163	best: 0.2732163 (1)	total: 13.2ms	remaining: 0us

bestTest = 0.2732162773
bestIteration = 1

0:	loss: 0.2732163	best: 0.2732163 (0)	total: 85.9ms	remaining: 6.1s
0:	learn: 0.3054717	test: 0.2850786	best: 0.2850786 (0)	total: 7.79ms	remaining: 23.4ms
1:	learn: 0.2979540	test: 0.2732163	best: 0.2732163 (1)	total: 13ms	remaining: 13ms
2:	learn: 0.2945489	test: 0.2719558	best: 0.2719558 (2)	total: 19.9ms	remaining: 6.64ms
3:	learn: 0.2917116	test: 0.2720258	best: 0.2719558 (2)	total: 24.9ms	remaining: 0us

bestTest = 0.2719557977
bestIteration = 2

1:	loss: 0.2719558	best: 0.2719558 (1)	total: 114ms	remaining: 3.98s
0:	learn: 0.3054717	test: 0.2850786	best: 0.2850786 (0)	total: 4.17ms	remaining: 20.9ms
1:	learn: 0.2979540	test: 0.2732163	best: 0.2732163 (1)	total: 9.69ms	remaining: 19.4ms
2:	learn: 0.2945489	test: 0.2719558	best: 0.2719558 (2)	total: 14.4ms	remaining:


bestTest = 0.2782866015
bestIteration = 7

12:	loss: 0.2782866	best: 0.2628175 (8)	total: 686ms	remaining: 3.11s
0:	learn: 0.3055954	test: 0.2850834	best: 0.2850834 (0)	total: 6.86ms	remaining: 61.7ms
1:	learn: 0.2940176	test: 0.2788686	best: 0.2788686 (1)	total: 13.2ms	remaining: 52.7ms
2:	learn: 0.2892368	test: 0.2794910	best: 0.2788686 (1)	total: 19.1ms	remaining: 44.5ms
3:	learn: 0.2875151	test: 0.2808600	best: 0.2788686 (1)	total: 26.1ms	remaining: 39.2ms
4:	learn: 0.2831634	test: 0.2808839	best: 0.2788686 (1)	total: 32.3ms	remaining: 32.3ms
5:	learn: 0.2821748	test: 0.2791945	best: 0.2788686 (1)	total: 47.3ms	remaining: 31.5ms
6:	learn: 0.2816896	test: 0.2792956	best: 0.2788686 (1)	total: 53.7ms	remaining: 23ms
7:	learn: 0.2789958	test: 0.2782866	best: 0.2782866 (7)	total: 59.2ms	remaining: 14.8ms
8:	learn: 0.2729607	test: 0.2806864	best: 0.2782866 (7)	total: 65.3ms	remaining: 7.26ms
9:	learn: 0.2714387	test: 0.2803261	best: 0.2782866 (7)	total: 73.4ms	remaining: 0us

bestTest =

5:	learn: 0.2802966	test: 0.2667998	best: 0.2651229 (2)	total: 50.5ms	remaining: 33.7ms
6:	learn: 0.2752589	test: 0.2663416	best: 0.2651229 (2)	total: 58.6ms	remaining: 25.1ms
7:	learn: 0.2743895	test: 0.2680505	best: 0.2651229 (2)	total: 65.8ms	remaining: 16.4ms
8:	learn: 0.2708596	test: 0.2661507	best: 0.2651229 (2)	total: 74.3ms	remaining: 8.25ms
9:	learn: 0.2692565	test: 0.2671941	best: 0.2651229 (2)	total: 81.4ms	remaining: 0us

bestTest = 0.2651228707
bestIteration = 2

22:	loss: 0.2651229	best: 0.2628175 (8)	total: 1.34s	remaining: 2.86s
0:	learn: 0.3054158	test: 0.2850338	best: 0.2850338 (0)	total: 9.32ms	remaining: 103ms
1:	learn: 0.2930405	test: 0.2671968	best: 0.2671968 (1)	total: 15.9ms	remaining: 79.8ms
2:	learn: 0.2872558	test: 0.2651229	best: 0.2651229 (2)	total: 22.8ms	remaining: 68.3ms
3:	learn: 0.2851340	test: 0.2691752	best: 0.2651229 (2)	total: 28.4ms	remaining: 56.9ms
4:	learn: 0.2814781	test: 0.2674518	best: 0.2651229 (2)	total: 33.9ms	remaining: 47.4ms
5:	learn: 

5:	learn: 0.2730751	test: 0.2655852	best: 0.2637707 (4)	total: 35.5ms	remaining: 23.7ms
6:	learn: 0.2677152	test: 0.2641125	best: 0.2637707 (4)	total: 54.3ms	remaining: 23.3ms
7:	learn: 0.2632991	test: 0.2653606	best: 0.2637707 (4)	total: 61.3ms	remaining: 15.3ms
8:	learn: 0.2577106	test: 0.2636222	best: 0.2636222 (8)	total: 68.6ms	remaining: 7.62ms
9:	learn: 0.2534700	test: 0.2645546	best: 0.2636222 (8)	total: 76.8ms	remaining: 0us

bestTest = 0.2636221625
bestIteration = 8

31:	loss: 0.2636222	best: 0.2628175 (8)	total: 1.97s	remaining: 2.46s
0:	learn: 0.3052594	test: 0.2844018	best: 0.2844018 (0)	total: 6.75ms	remaining: 74.3ms
1:	learn: 0.2887594	test: 0.2649816	best: 0.2649816 (1)	total: 14.2ms	remaining: 71ms
2:	learn: 0.2842322	test: 0.2657602	best: 0.2649816 (1)	total: 22.4ms	remaining: 67.3ms
3:	learn: 0.2809998	test: 0.2667286	best: 0.2649816 (1)	total: 29.1ms	remaining: 58.2ms
4:	learn: 0.2742318	test: 0.2637707	best: 0.2637707 (4)	total: 35.1ms	remaining: 49.1ms
5:	learn: 0

7:	learn: 0.2407606	test: 0.2819212	best: 0.2619317 (3)	total: 58.5ms	remaining: 0us

bestTest = 0.2619316837
bestIteration = 3

39:	loss: 0.2619317	best: 0.2619317 (37)	total: 2.56s	remaining: 2.04s
0:	learn: 0.3030295	test: 0.2753589	best: 0.2753589 (0)	total: 10.8ms	remaining: 96.9ms
1:	learn: 0.2893777	test: 0.2652289	best: 0.2652289 (1)	total: 21.2ms	remaining: 84.8ms
2:	learn: 0.2874813	test: 0.2672078	best: 0.2652289 (1)	total: 27.5ms	remaining: 64.3ms
3:	learn: 0.2815239	test: 0.2619317	best: 0.2619317 (3)	total: 35.3ms	remaining: 53ms
4:	learn: 0.2733757	test: 0.2729644	best: 0.2619317 (3)	total: 47.4ms	remaining: 47.4ms
5:	learn: 0.2612121	test: 0.2841011	best: 0.2619317 (3)	total: 55.5ms	remaining: 37ms
6:	learn: 0.2493943	test: 0.2767013	best: 0.2619317 (3)	total: 63.7ms	remaining: 27.3ms
7:	learn: 0.2407606	test: 0.2819212	best: 0.2619317 (3)	total: 70.7ms	remaining: 17.7ms
8:	learn: 0.2400795	test: 0.2821365	best: 0.2619317 (3)	total: 79.9ms	remaining: 8.88ms
9:	learn: 0.

2:	learn: 0.2716775	test: 0.2724092	best: 0.2693161 (1)	total: 41.1ms	remaining: 95.9ms
3:	learn: 0.2696953	test: 0.2723796	best: 0.2693161 (1)	total: 46.8ms	remaining: 70.2ms
4:	learn: 0.2603100	test: 0.2688811	best: 0.2688811 (4)	total: 62ms	remaining: 62ms
5:	learn: 0.2524717	test: 0.2770420	best: 0.2688811 (4)	total: 72.6ms	remaining: 48.4ms
6:	learn: 0.2322559	test: 0.2864417	best: 0.2688811 (4)	total: 87.8ms	remaining: 37.6ms
7:	learn: 0.2310593	test: 0.2862448	best: 0.2688811 (4)	total: 94.5ms	remaining: 23.6ms
8:	learn: 0.2171527	test: 0.2985971	best: 0.2688811 (4)	total: 108ms	remaining: 12ms
9:	learn: 0.2155228	test: 0.2964352	best: 0.2688811 (4)	total: 117ms	remaining: 0us

bestTest = 0.2688811411
bestIteration = 4

49:	loss: 0.2688811	best: 0.2619317 (37)	total: 3.45s	remaining: 1.52s
0:	learn: 0.3025426	test: 0.2770115	best: 0.2770115 (0)	total: 7.86ms	remaining: 86.5ms
1:	learn: 0.2845899	test: 0.2693161	best: 0.2693161 (1)	total: 16.5ms	remaining: 82.5ms
2:	learn: 0.2716

4:	learn: 0.2418130	test: 0.2839870	best: 0.2716211 (2)	total: 96ms	remaining: 96ms
5:	learn: 0.2413316	test: 0.2836083	best: 0.2716211 (2)	total: 106ms	remaining: 70.6ms
6:	learn: 0.2333818	test: 0.2884640	best: 0.2716211 (2)	total: 125ms	remaining: 53.6ms
7:	learn: 0.2214613	test: 0.2897134	best: 0.2716211 (2)	total: 137ms	remaining: 34.2ms
8:	learn: 0.2154602	test: 0.2910027	best: 0.2716211 (2)	total: 150ms	remaining: 16.7ms
9:	learn: 0.1903147	test: 0.2986234	best: 0.2716211 (2)	total: 161ms	remaining: 0us

bestTest = 0.2716210718
bestIteration = 2

58:	loss: 0.2716211	best: 0.2619317 (37)	total: 4.7s	remaining: 1.04s
0:	learn: 0.2959857	test: 0.2719731	best: 0.2719731 (0)	total: 10.9ms	remaining: 120ms
1:	learn: 0.2804982	test: 0.2718807	best: 0.2718807 (1)	total: 21.8ms	remaining: 109ms
2:	learn: 0.2727241	test: 0.2716211	best: 0.2716211 (2)	total: 32.6ms	remaining: 97.9ms
3:	learn: 0.2530468	test: 0.2745412	best: 0.2716211 (2)	total: 43.3ms	remaining: 86.7ms
4:	learn: 0.2418130	

3:	learn: 0.2447660	test: 0.2778772	best: 0.2726710 (1)	total: 74.1ms	remaining: 111ms
4:	learn: 0.2423893	test: 0.2769558	best: 0.2726710 (1)	total: 84.6ms	remaining: 84.6ms
5:	learn: 0.2394725	test: 0.2778302	best: 0.2726710 (1)	total: 134ms	remaining: 89.6ms
6:	learn: 0.2378856	test: 0.2768101	best: 0.2726710 (1)	total: 149ms	remaining: 63.7ms
7:	learn: 0.2225024	test: 0.2835620	best: 0.2726710 (1)	total: 172ms	remaining: 43ms
8:	learn: 0.2224071	test: 0.2836416	best: 0.2726710 (1)	total: 185ms	remaining: 20.6ms
9:	learn: 0.2196037	test: 0.2861981	best: 0.2726710 (1)	total: 207ms	remaining: 0us

bestTest = 0.2726709855
bestIteration = 1

67:	loss: 0.2726710	best: 0.2619317 (37)	total: 6.25s	remaining: 368ms
0:	learn: 0.2917105	test: 0.2778895	best: 0.2778895 (0)	total: 16.7ms	remaining: 184ms
1:	learn: 0.2641523	test: 0.2726710	best: 0.2726710 (1)	total: 34.7ms	remaining: 174ms
2:	learn: 0.2511529	test: 0.2786740	best: 0.2726710 (1)	total: 53ms	remaining: 159ms
3:	learn: 0.2447660	t

In [142]:
grid_search_result['params']

{'depth': 6, 'iterations': 4}

In [62]:
model = CatBoostClassifier(iterations=4,
                           depth = 6,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [63]:
model.fit(train_pool)

0:	learn: 0.3266170	total: 7.29ms	remaining: 21.9ms
1:	learn: 0.2858492	total: 13.5ms	remaining: 13.5ms
2:	learn: 0.2768046	total: 20.4ms	remaining: 6.8ms
3:	learn: 0.2744963	total: 25.1ms	remaining: 0us


In [65]:
accuracy_score(y_test, model.predict(test_pool))

0.904

In [66]:
df.drop('y', axis = 1).columns.to_list()

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m']

In [67]:
df_test = pd.read_csv('/home/max/Documents/test.csv', sep = ';', names =  ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'x', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m'], index_col= False)
# df_test[columns_list]
df_test

/tmp/ipykernel_8340/2248158909.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_test = pd.read_csv('/home/max/Documents/test.csv', sep = ';', names =  ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,x,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m
0,49,management,divorced,university.degree,no,no,no,cellular,jul,thu,144,5,999,0,nonexistent,1.4,93.918,-42.7,4.963
1,31,admin.,unknown,high.school,no,no,no,telephone,may,mon,182,8,999,0,nonexistent,1.1,93.994,-36.4,4.858
2,31,admin.,single,basic.9y,no,no,no,telephone,may,wed,80,1,999,1,failure,-1.8,92.893,-46.2,1.281
3,60,admin.,married,high.school,no,yes,yes,cellular,apr,thu,482,2,999,0,nonexistent,-1.8,93.075,-47.1,1.365
4,33,blue-collar,married,high.school,no,no,no,cellular,apr,mon,87,2,999,2,failure,-1.8,93.075,-47.1,1.405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,53,1,999,0,nonexistent,1.4,93.918,-42.7,4.958
1116,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,219,1,999,0,nonexistent,1.4,93.918,-42.7,4.959
1117,27,student,single,high.school,no,no,no,cellular,may,mon,64,2,999,1,failure,-1.8,92.893,-46.2,1.354
1118,58,admin.,married,high.school,no,no,no,cellular,aug,fri,528,1,999,0,nonexistent,1.4,93.444,-36.1,4.966


In [68]:
# df_test.columns = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
#        'contact', 'month', 'day_of_week', 'x', 'campaign', 'pdays', 'previous',
#        'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
#        'euribor3m']
# , 'nr.employed'
df_test.drop('x', axis = 1, inplace = True)

In [69]:
sub_pool = Pool(df_test, cat_features=df_test.columns[df_test.dtypes == 'object'].to_list()) 

In [70]:
df_ans = pd.DataFrame(model.predict(sub_pool), columns = ['prediction'])
df_ans.index += 1
df_ans

,prediction
1,0
2,0
3,0
4,0
5,0
...,...
1116,0
1117,0
1118,0
1119,0


In [72]:
df_ans.to_csv('/home/max/Documents/marketing_submission.csv', index_label = 'Id')

In [ ]:
'''
-only numeric features
+gridsearch catboost
-get dummies all and feature selection
'''

In [143]:
df = pd.read_csv('/home/max/Documents/train.csv', sep = ';')

In [144]:
df.drop(df.columns[15:20], axis = 1, inplace = True)

In [145]:
df.drop('duration', axis = 1, inplace = True)

In [146]:
df = pd.get_dummies(df)

In [147]:
df.rename(columns = {'y_yes': 'y'}, inplace = True)

In [148]:
df.drop('y_no', axis = 1, inplace = True)

In [149]:
# fig, axes = plt.subplots(2, figsize = (10,16))
# #add DataFrames to subplots
# df_yes.age.plot(kind = 'hist', ax=axes[0])
# axes[0].set_xlabel('age')
# axes[0].grid()
# axes[0].set_axisbelow(True)
# df_no.age.plot(kind = 'hist', ax=axes[1])
# axes[1].grid()
# axes[1].set_axisbelow(True)
# axes[1].set_xlabel('age')

In [151]:
X, y = df.drop('y', axis = 1), df.y

In [152]:
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.ensemble import RandomForestClassifier

In [153]:
clf = RandomForestClassifier()

In [154]:
params = {'n_estimators':range(5,30, 5), 'max_depth': range(1,20)}

In [155]:
grid_search_cv_clf = GridSearchCV(clf, params, cv = 5)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [157]:
grid_search_cv_clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 20),
                         'n_estimators': range(5, 30, 5)})

In [158]:
grid_search_cv_clf.best_params_

{'max_depth': 6, 'n_estimators': 25}

In [159]:
best_clf = grid_search_cv_clf.best_estimator_

In [160]:
accuracy_score(y_test, best_clf.predict(X_test))

0.896

In [172]:
df_test = pd.read_csv('/home/max/Documents/test.csv', sep = ';', names =  ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'x', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m'], index_col= False)
# df_test[columns_list]
df_test.drop(['x', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m'], axis = 1, inplace = True)

/tmp/ipykernel_8340/945682200.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_test = pd.read_csv('/home/max/Documents/test.csv', sep = ';', names =  ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',


In [173]:
df_test = pd.get_dummies(df_test)

In [174]:
df_test.drop(['default_yes', 'education_illiterate'], axis = 1, inplace = True)

In [175]:
df_ans = pd.DataFrame(best_clf.predict(df_test), columns = ['prediction'])
df_ans.index += 1
df_ans

,prediction
1,0
2,0
3,0
4,0
5,0
...,...
1116,0
1117,0
1118,0
1119,0


In [176]:
df_ans.to_csv('/home/max/Documents/marketing_submission_rf.csv', index_label = 'Id')